In [3]:
# these warnings are fine. you can ignore them.
import random, math

import sys
sys.path.append('../')
from util import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Imports finished.")

Imports finished.


# Setting Up Dataset/Model/Ground Truth

In [4]:
dataset = Dataset(root='/tmp/CiteSeer', name='CiteSeer', device=device)
data, in_feats, h_feats, num_classes = dataset.get_data()

model = get_model(in_feats, h_feats, num_classes, 'citeseer')

ground_truth = get_ground_truth(model, data)
print(ground_truth)

0.537


# Experiments
**Note:** The ground truth here is $0.749$

In [5]:
homophilic_set = {label: [] for label in set(data.y.tolist())}

test_indices = torch.nonzero(data.test_mask, as_tuple=False).squeeze()
for i in test_indices:
    homophilic_set[data.y[i].item()].append(i.item())

# print("This is the dictionary containing each class and its respective elements:\n\t", homophilic_set)

In [18]:
def calculate_homophily(data):
    G, x, y, train_mask, test_mask = convert_to_networkx(data)
    same = 0
    num_edges = 0
    
    for i in range(0, G.number_of_nodes()):
        edges = G.out_edges(i)
        for e in edges:
            if y[e[0]] == y[e[1]]:
                same += 1
            num_edges += 1

    return same / num_edges

homo = calculate_homophily(data)
print(homo)

0.7355008787346221


### Connect with same class nodes which do not already have edges
Take the first element in a class and add edges to all OTHER elements in that class if they do not currently exist.

In [6]:
data = dataset.get_data()[0]
modified_graph = data

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

for i in range(0, num_classes):
    for j in range(1, len(homophilic_set[i])):
        if not G.has_edge(i, j):
            add_edge(G, i, j, undirected=True)

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph)) 
number_added_edges(init_edges, final_edges, is_undirected=True)


----
The accuracy has changed by -0.0010
Change in edges:  980.5  | Percentage change: 21.54%


### Connect same class nodes with one other random nodes (which is is not currently a neighbor to)

In [7]:
data = dataset.get_data()[0]
modified_graph = data

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

for i in range(0, num_classes):
    r_node = random.choice(homophilic_set[i])
    n_node = random.choice(homophilic_set[i])

    while r_node == n_node or G.has_edge(r_node, n_node):
        n_node = random.choice(homophilic_set[i])

    add_edge(G, r_node, n_node, undirected=True)

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph)) 
number_added_edges(init_edges, final_edges, is_undirected=True)


----
The accuracy has changed by 0.0020
Change in edges:  6.0  | Percentage change: 0.13%


### Create a dense graph between all nodes with the same class

In [8]:
data = dataset.get_data()[0]
modified_graph = data

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        for k in range(j + 1, len(homophilic_set[i])):
            j_node = homophilic_set[i][j]
            k_node = homophilic_set[i][k]
            if j_node == k_node or G.has_edge(j_node, k_node):
                continue

            add_edge(G, j_node, k_node, undirected=True)

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)


----
The accuracy has changed by 0.4620
Change in edges:  88856.0  | Percentage change: 1952.02%


#### Increase each node's number homophilic edges by a certain threshold?
Let's start with $\lfloor 0.1 \times h_e \rfloor$, where $h_e$ is the number of homophilic edges.
In the current implementation, the nodes which are seen last likely see more nodes added.

In [9]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.1

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has not changed.
Change in edges:  0.0  | Percentage change: 0.00%


In [10]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.15

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has not changed.
Change in edges:  2.0  | Percentage change: 0.04%


In [11]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.20

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has changed by 0.0020
Change in edges:  10.0  | Percentage change: 0.22%


In [12]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.25

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has changed by -0.0010
Change in edges:  14.0  | Percentage change: 0.31%


In [13]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.3

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has changed by 0.0030
Change in edges:  16.0  | Percentage change: 0.35%


In [14]:
data = dataset.get_data()[0]
modified_graph = data
c = 1/3

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has changed by 0.0120
Change in edges:  60.0  | Percentage change: 1.32%


In [15]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.35

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has changed by 0.0080
Change in edges:  60.0  | Percentage change: 1.32%


In [16]:
data = dataset.get_data()[0]
modified_graph = data
c = 0.5

init_edges = len(modified_graph.edge_index[1])

G, x, y, train_mask, test_mask = convert_to_networkx(modified_graph)

# class_and_added = {label: [] for label in set(g.ndata["label"].tolist())}

# note this implementation is likely inefficient
for i in range(0, num_classes):
    for j in range(0, len(homophilic_set[i])):
        edges = G.out_edges(homophilic_set[i][j])

        same_class_edges = set()
        for e in edges:
            if (e[1] in homophilic_set[i]):
                same_class_edges.add(e[1])

        for k in range(0, math.floor(len(same_class_edges) * c)):
            b_node = homophilic_set[i][j]
            c_node = random.choice(homophilic_set[i])

            ctr = 0
            while (b_node == c_node or G.has_edge(b_node, c_node)) and ctr <= math.floor(len(same_class_edges) * c):
                c_node = random.choice(homophilic_set[i])
                ctr += 1

            add_edge(G, b_node, c_node, undirected=True)
            # class_and_added[i].append(len(same_class_edges))

modified_graph = convert_to_pyg(G, x, y, train_mask, test_mask)
final_edges = len(modified_graph.edge_index[1])

output_accuracy_change(ground_truth, test_model(model, modified_graph))
number_added_edges(init_edges, final_edges, is_undirected=True)

# print(class_and_added)


----
The accuracy has changed by 0.0280
Change in edges:  166.0  | Percentage change: 3.65%
